## Sentiment Analysis
The process of computationally identifying and categorizing opinions expressed in a piece of text, especially in order to determine whether the writer's attitude towards a particular topic is positive, negative, or neutral.

In [1]:
import numpy as np
import pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix,classification_report
import re

Reading the data and keeping only two column require for analysis.

In [2]:
data = pd.read_csv('/content/drive/MyDrive/Sentiment.csv')
data = data[['text','sentiment']]

In [3]:
data.head(10)

,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive
5,"RT @GregAbbott_TX: @TedCruz: ""On my first day ...",Positive
6,RT @warriorwoman91: I liked her and was happy ...,Negative
7,Going on #MSNBC Live with @ThomasARoberts arou...,Neutral
8,Deer in the headlights RT @lizzwinstead: Ben C...,Negative
9,RT @NancyOsborne180: Last night's debate prove...,Negative


## DATA PREPROCESSING

1. Filtering out Neutral Tweets
2. converting text into lowercase
3. Using regular expression to remove any special characters

In [ ]:
data = data[data.sentiment != "Neutral"]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

data.head()

,text,sentiment
1,rt scottwalker didnt catch the full gopdebate ...,Positive
3,rt robgeorge that carly fiorina is trending h...,Positive
4,rt danscavino gopdebate w realdonaldtrump deli...,Positive
5,rt gregabbott_tx tedcruz on my first day i wil...,Positive
6,rt warriorwoman91 i liked her and was happy wh...,Negative


In [ ]:
print("Number of Positive tweets",data[ data['sentiment'] == 'Positive'].size)
print("Number of Negative tweets",data[ data['sentiment'] == 'Negative'].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt','')
data.head()

Number of Positive tweets 4472
Number of Negative tweets 16986


,text,sentiment
1,scottwalker didnt catch the full gopdebate la...,Positive
3,robgeorge that carly fiorina is trending hou...,Positive
4,danscavino gopdebate w realdonaldtrump delive...,Positive
5,gregabbott_tx tedcruz on my first day i will ...,Positive
6,warriorwoman91 i liked her and was happy when...,Negative


### Tokenise and padding of text

In [ ]:
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)
X[:2]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
         359,  120,    1,  692,    2,   39,   58,  234,   37,  207,    6,
         172, 1745,   12, 1308, 1394,  733],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          16,  281,  249,    5,  809,  102,  170,   26,  134,    6,    1,
         171,   12,    2,  231,  713,   17]], dtype=int32)

 The Embedding layer plays a crucial role in converting discrete words into continuous vector representations that capture semantic relationships.

 2000 * 128+0+(4×(128×196+196))+(196×2+2)=256000+254800+394=511194

In [ ]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.4, recurrent_dropout=0.4))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 28, 128)           256000    
                                                                 
 spatial_dropout1d (Spatial  (None, 28, 128)           0         
 Dropout1D)                                                      
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 2)                 394       
                                                                 
Total params: 511194 (1.95 MB)
Trainable params: 511194 (1.95 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(8583, 28) (8583, 2)
(2146, 28) (2146, 2)


In [ ]:
batch_size = 128
model.fit(X_train, Y_train, epochs = 15, batch_size=batch_size, verbose = 1)

Epoch 1/15
68/68 [==============================] - 44s 538ms/step - loss: 0.4974 - accuracy: 0.7904
Epoch 2/15
68/68 [==============================] - 30s 444ms/step - loss: 0.3522 - accuracy: 0.8499
Epoch 3/15
68/68 [==============================] - 23s 341ms/step - loss: 0.2959 - accuracy: 0.8735
Epoch 4/15
68/68 [==============================] - 22s 321ms/step - loss: 0.2750 - accuracy: 0.8875
Epoch 5/15
68/68 [==============================] - 37s 544ms/step - loss: 0.2569 - accuracy: 0.8901
Epoch 6/15
68/68 [==============================] - 32s 470ms/step - loss: 0.2427 - accuracy: 0.8986
Epoch 7/15
68/68 [==============================] - 30s 445ms/step - loss: 0.2389 - accuracy: 0.9021
Epoch 8/15
68/68 [==============================] - 23s 333ms/step - loss: 0.2221 - accuracy: 0.9059
Epoch 9/15
68/68 [==============================] - 23s 333ms/step - loss: 0.2114 - accuracy: 0.9090
Epoch 10/15
68/68 [==============================] - 23s 342ms/step - loss: 0.1992 - accura

In [ ]:
y_pred_prob = model.predict(X_test, batch_size=batch_size)
y_pred = np.argmax(y_pred_prob, axis=1)

17/17 [==============================] - 1s 62ms/step


In [ ]:
df_test = pd.DataFrame({'true': Y_test.tolist(), 'pred':y_pred})
df_test['true'] = df_test['true'].apply(lambda x: np.argmax(x))
print("confusion matrix ")
print(confusion_matrix(df_test.true, df_test.pred))
print(classification_report(df_test.true, df_test.pred))

confusion matrix 
[[1572  141]
 [ 202  231]]
              precision    recall  f1-score   support

           0       0.89      0.92      0.90      1713
           1       0.62      0.53      0.57       433

    accuracy                           0.84      2146
   macro avg       0.75      0.73      0.74      2146
weighted avg       0.83      0.84      0.84      2146



Precision is if a model predict it as negative/positive then it is +ve/-ve.
Recall is how many of total poituve/negative correctly classify as that.

1. The model has a relatively high precision for negative sentiment (0), meaning that when it predicts a negative sentiment, it's often correct.
2. The recall for positive sentiment (1) is moderate, indicating that the model identifies only about 53% of actual positive sentiment instances.

In [ ]:
data_majority = data[data['sentiment'] == 'Negative']
data_minority = data[data['sentiment'] == 'Positive']

bias = data_minority.shape[0]/data_majority.shape[0]
train = pd.concat([data_majority.sample(frac=0.8,random_state=200),data_minority.sample(frac=0.8,random_state=200)])
test = pd.concat([data_majority.drop(data_majority.sample(frac=0.8,random_state=200).index),data_minority.drop(data_minority.sample(frac=0.8,random_state=200).index)])

train = shuffle(train)
test = shuffle(test)

In [ ]:
print('positive data in training:',(train.sentiment == 'Positive').sum())
print('negative data in training:',(train.sentiment == 'Negative').sum())
print('positive data in test:',(test.sentiment == 'Positive').sum())
print('negative data in test:',(test.sentiment == 'Negative').sum())

positive data in training: 1789
negative data in training: 6794
positive data in test: 447
negative data in test: 1699


In [ ]:
data_majority = train[train['sentiment'] == 'Negative']
data_minority = train[train['sentiment'] == 'Positive']

print("majority class before upsample:",data_majority.shape)
print("minority class before upsample:",data_minority.shape)

data_minority_upsampled = resample(data_minority,replace=True,n_samples= data_majority.shape[0],random_state=123)

data_upsampled = pd.concat([data_majority, data_minority_upsampled])

print("After upsampling\n",data_upsampled.sentiment.value_counts(),sep = "")

max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['text'].values)

X_train = tokenizer.texts_to_sequences(data_upsampled['text'].values)
X_train = pad_sequences(X_train,maxlen=29)
Y_train = pd.get_dummies(data_upsampled['sentiment']).values
print('x_train shape:',X_train.shape)

X_test = tokenizer.texts_to_sequences(test['text'].values)
X_test = pad_sequences(X_test,maxlen=29)
Y_test = pd.get_dummies(test['sentiment']).values
print("x_test shape", X_test.shape)

majority class before upsample: (6794, 2)
minority class before upsample: (1789, 2)
After upsampling
Negative    6794
Positive    6794
Name: sentiment, dtype: int64
x_train shape: (13588, 29)
x_test shape (2146, 29)


In [ ]:
embed_dim = 128
lstm_out = 192

model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X_train.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.4, recurrent_dropout=0.4))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 29, 128)           256000    
                                                                 
 spatial_dropout1d_1 (Spati  (None, 29, 128)           0         
 alDropout1D)                                                    
                                                                 
 lstm_1 (LSTM)               (None, 192)               246528    
                                                                 
 dense_1 (Dense)             (None, 2)                 386       
                                                                 
Total params: 502914 (1.92 MB)
Trainable params: 502914 (1.92 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
batch_size = 128
class_weights = {0: 1 ,1: 1.6/bias }
model.fit(X_train, Y_train, epochs = 15, batch_size=batch_size, verbose = 1,class_weight=class_weights)

Epoch 1/15
107/107 [==============================] - 41s 351ms/step - loss: 1.2010 - accuracy: 0.5941
Epoch 2/15
107/107 [==============================] - 34s 322ms/step - loss: 0.7357 - accuracy: 0.7770
Epoch 3/15
107/107 [==============================] - 34s 322ms/step - loss: 0.6073 - accuracy: 0.8288
Epoch 4/15
107/107 [==============================] - 35s 326ms/step - loss: 0.5291 - accuracy: 0.8532
Epoch 5/15
107/107 [==============================] - 37s 340ms/step - loss: 0.4779 - accuracy: 0.8675
Epoch 6/15
107/107 [==============================] - 34s 321ms/step - loss: 0.4431 - accuracy: 0.8837
Epoch 7/15
107/107 [==============================] - 34s 322ms/step - loss: 0.4089 - accuracy: 0.8939
Epoch 8/15
107/107 [==============================] - 42s 396ms/step - loss: 0.3745 - accuracy: 0.9012
Epoch 9/15
107/107 [==============================] - 35s 324ms/step - loss: 0.3619 - accuracy: 0.9089
Epoch 10/15
107/107 [==============================] - 37s 344ms/step - l

In [ ]:
y_pred_prob = model.predict(X_test, batch_size=batch_size)
y_pred = np.argmax(y_pred_prob, axis=1)

df_test = pd.DataFrame({'true': Y_test.tolist(), 'pred':y_pred})
df_test['true'] = df_test['true'].apply(lambda x: np.argmax(x))
print("confusion matrix")
print(confusion_matrix(df_test.true, df_test.pred))
print(classification_report(df_test.true, df_test.pred))

17/17 [==============================] - 2s 101ms/step
confusion matrix
[[1337  362]
 [ 107  340]]
              precision    recall  f1-score   support

           0       0.93      0.79      0.85      1699
           1       0.48      0.76      0.59       447

    accuracy                           0.78      2146
   macro avg       0.71      0.77      0.72      2146
weighted avg       0.83      0.78      0.80      2146



In [ ]:
twt = ['keep up the good work']

twt = tokenizer.texts_to_sequences(twt)

twt = pad_sequences(twt, maxlen=29, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0 381  47   1 137 464]]
1/1 - 0s - 28ms/epoch - 28ms/step
positive


In [ ]:
twt = ['DISAPPOINTED! BAD WORK ']

twt = tokenizer.texts_to_sequences(twt)

twt = pad_sequences(twt, maxlen=29, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0 604 343 464]]
1/1 - 0s - 27ms/epoch - 27ms/step
negative


In [ ]:
twt = ['This film is terrible!']

twt = tokenizer.texts_to_sequences(twt)

twt = pad_sequences(twt, maxlen=29, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0  20   5 988]]
1/1 - 0s - 27ms/epoch - 27ms/step
negative


In [ ]:
twt = ['This film is great!']

twt = tokenizer.texts_to_sequences(twt)

twt = pad_sequences(twt, maxlen=29, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0  20   5 144]]
1/1 - 0s - 47ms/epoch - 47ms/step
positive


In [ ]:
twt = ["This film is not great, it's terrible"]

twt = tokenizer.texts_to_sequences(twt)

twt = pad_sequences(twt, maxlen=29, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0  20   5  19 144 988]]
1/1 - 0s - 29ms/epoch - 29ms/step
negative


In [ ]:
twt = ["Animal is a completely misogynist film"]

twt = tokenizer.texts_to_sequences(twt)

twt = pad_sequences(twt, maxlen=29, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    5    7  888
  1542]]
1/1 - 0s - 28ms/epoch - 28ms/step
negative


## Hyper Parameter training

In [ ]:
# !pip install scikeras

In [ ]:
# from sklearn.model_selection import GridSearchCV
# from keras.models import Sequential
# from keras.layers import Dense
# from scikeras.wrappers import KerasClassifier

In [ ]:
# # Function to create model, required for KerasClassifier
# def create_model(dropout_rate = 0.0):
#     # create model
#     embed_dim = 128
#     lstm_out = 192
#     model = Sequential()
#     model.add(Embedding(max_features, embed_dim,input_length = X_train.shape[1]))
#     model.add(SpatialDropout1D(dropout_rate))
#     model.add(LSTM(lstm_out, dropout=dropout_rate, recurrent_dropout=dropout_rate))
#     model.add(Dense(2,activation='softmax'))
#     model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
# #     print(model.summary())
#     return model
# # fix random seed for reproducibility
# seed = 7
# np.random.seed(seed)

# model = KerasClassifier(build_fn=create_model, verbose=2,epochs=30, batch_size=128,dropout_rate=0.0)
# # define the grid search parameters
# # batch_size = [128]
# # epochs = [5]
# dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
# # class_weight = [{0: 1, 1: 1/bias},{0: 1, 1: 1.2/bias},{0: 1, 1: 1.5/bias},{0: 1, 1: 1.8/bias}]
# param_grid = dict(dropout_rate = dropout_rate)
# grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
# grid_result = grid.fit(X_train, Y_train)
# # summarize results
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))